In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.now())
print(f"{current_date}")

2025-02-17 21:24:58.385205


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date =current_date.floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-02-17T21:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-02-17 21:00:00')

In [8]:
fetch_data_from

Timestamp('2025-01-19 21:00:00')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-01.
Loading data for 2024-01...


Total records: 2,964,624
Valid records: 2,911,483
Records dropped: 53,141 (1.79%)
Successfully processed data for 2024-01.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id
2273352,2025-02-12 16:25:44,2
2580053,2025-02-15 16:56:40,2
947469,2025-01-20 07:53:24,3
27030,2025-01-20 10:33:31,3
31360,2025-01-20 11:40:12,3
...,...,...
2763907,2025-02-17 20:55:03,263
2762251,2025-02-17 20:55:17,263
2762595,2025-02-17 20:56:28,263
2763271,2025-02-17 20:58:38,263


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-19 21:00:00,2,0
1,2025-01-19 22:00:00,2,0
2,2025-01-19 23:00:00,2,0
3,2025-01-20 00:00:00,2,0
4,2025-01-20 01:00:00,2,0
...,...,...,...
176083,2025-02-17 16:00:00,263,70
176084,2025-02-17 17:00:00,263,82
176085,2025-02-17 18:00:00,263,102
176086,2025-02-17 19:00:00,263,84


In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176088 entries, 0 to 176087
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         176088 non-null  datetime64[ns]
 1   pickup_location_id  176088 non-null  int16         
 2   rides               176088 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [15]:
ts_data.tail()

,pickup_hour,pickup_location_id,rides
176083,2025-02-17 16:00:00,263,70
176084,2025-02-17 17:00:00,263,82
176085,2025-02-17 18:00:00,263,102
176086,2025-02-17 19:00:00,263,84
176087,2025-02-17 20:00:00,263,71


In [16]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176088 entries, 0 to 176087
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         176088 non-null  datetime64[ns]
 1   pickup_location_id  176088 non-null  int16         
 2   rides               176088 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [17]:
ts_data.tail()

,pickup_hour,pickup_location_id,rides
176083,2025-02-17 16:00:00,263,70
176084,2025-02-17 17:00:00,263,82
176085,2025-02-17 18:00:00,263,102
176086,2025-02-17 19:00:00,263,84
176087,2025-02-17 20:00:00,263,71


In [18]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')

# Initialize connection to Hopsworks
project = hopsworks.login(
    api_key_value=api_key,
    project=project_name
)
print(f"Successfully connected to Hopsworks project: {project_name}")

c:\Users\nolan\anaconda3\envs\sp25_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-02-17 21:25:08,012 INFO: Initializing external client
2025-02-17 21:25:08,014 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 21:25:09,453 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Successfully connected to Hopsworks project: nolanphi


In [19]:
feature_store = project.get_feature_store()

In [20]:
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

In [21]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 176088/176088 | Elapsed Time: 00:12 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)